In [ ]:
import datetime as dt
import numpy as np
from scipy.stats import poisson
import matplotlib.pyplot as plt
from define.randchans import RandomUnitary
from define.QECCLfid import minwt as mw
from define import qcode as qc
from define import fnames as fn
from define import globalvars as gv
import matplotlib
matplotlib.use("Agg")
from matplotlib.backends.backend_pdf import PdfPages

In [ ]:
qcode = qc.QuantumErrorCorrectingCode("Steane")
qc.Load(qcode)

In [ ]:
mu = np.mean(qcode.weightdist[qcode.PauliCorrectableIndices])/2
np.random.normal(mu,0.5)

In [ ]:
# plt.plot(np.arange(8),poisson.pmf(np.arange(8), 0),0.1**np.arange(10),marker="o",linestyle="None")

In [ ]:
# probs = poisson.pmf(qcode.weightdist, np.abs(np.random.normal(mu,mu/3)))
# plt.plot(np.sort(qcode.weightdist), probs[np.argsort(qcode.weightdist)],marker="o",linestyle="None")
# plt.plot(np.sort(qcode.weightdist), (1/7)**np.sort(qcode.weightdist),marker="x",linestyle="None")
# plt.yscale('log')

In [ ]:
errdist = RandomPauliChannel({"method":1, "mu":mu, "weightdist":qcode.weightdist})

In [ ]:
infid = 0.2
errdist[0] = 1 - infid
errdist[1:] = infid * errdist[1:]/np.sum(errdist[1:])

In [ ]:
errdist

In [ ]:
PauliDistributionPlot(errdist, qcode, nreps=5, max_weight=3)

In [ ]:
for w in range(np.max(qcode.weightdist)+1):
    print("Total probability of weight {} errors = {}:\n{}".format(w, np.sum(errdist[qcode.weightdist==w]), errdist[qcode.weightdist==w]))

In [ ]:
import numpy as np
from define import qcode as qc
from define.QECCLfid import uncorrectable as uc
from analyze.plots import PauliDistributionPlot

In [ ]:
qcode = qc.QuantumErrorCorrectingCode("Steane")
qc.Load(qcode)
qc.PrepareSyndromeLookUp(qcode)

In [ ]:
q1 = 0.8; q2 = 0.2; infid = 0.1; n = qcode.N
single_qubit_errors = np.array([1 - infid, infid/3, infid/3, infid/3], dtype=np.double)

In [ ]:
iid_error_dist = uc.GetErrorProbabilities(qcode.PauliOperatorsLST, single_qubit_errors, 0)
full_process_infid = 1 - iid_error_dist[0]
print("Sum of IID error probabilities = {}, Infidelity = {}.".format(np.sum(iid_error_dist), full_process_infid))

In [ ]:
n_two_qubit_errors = np.int(0.1 * qcode.group_by_weight[2].size)
two_qubit_errors = np.random.choice(qcode.group_by_weight[2], size=n_two_qubit_errors)

In [ ]:
corr_error_dist = np.zeros(iid_error_dist.size, dtype=np.double)
corr_error_dist[two_qubit_errors] = np.abs(np.random.normal(0.1 * 4**n * full_process_infid, 0.1 * 4**n * full_process_infid, size=(n_two_qubit_errors,)))
corr_error_dist[0] = 1 - full_process_infid
corr_error_dist[two_qubit_errors] = full_process_infid * corr_error_dist[two_qubit_errors]/np.sum(corr_error_dist[two_qubit_errors])
corr_error_dist = corr_error_dist/np.sum(corr_error_dist)

In [ ]:
print("Sum of CORR error probabilities = {}, Infidelity = {}".format(np.sum(corr_error_dist), 1-corr_error_dist[0]))

In [ ]:
pauli_error_dist = q1 * iid_error_dist + q2 * corr_error_dist

In [ ]:
PauliDistributionPlot(qcode, pauli_error_dist, nreps=5, max_weight=3,outdir="./../../temp/", channel="linear_sum")

In [ ]:
chi_from_file = np.load("/Users/pavi/Documents/chbank/20_05_2020_11_07_46/physical/raw_up_6.5.npy")[1,:].reshape([4,4])

In [ ]:
chi_from_file

In [ ]:
ptm_from_file = np.load("/Users/pavi/Documents/chbank/20_05_2020_11_07_46/physical/up_6.5.npy")[1,:].reshape([4,4])

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
def normalization(p, n, g):
    return (1 + g*p + np.power(p,3)/(1-p)*(1-np.power(p,n-2)))

In [ ]:
nq = 7
g = 0.1
p = 0.1
corr_probs = [(1-p)/normalization(1-p, nq, g)] + [p/normalization(p, nq, g), g*p/normalization(p, nq, g)] + [np.power(p,k)/normalization(p, nq, g) for k in range(3,nq+1)]

In [ ]:
iid_probs = [np.power(1-p,nq)] + [np.power(p,k)*np.power(1-p, nq-k) for k in range(1, nq+1)]

In [ ]:
#fig = plt.figure()
X = np.arange(nq+1)
plt.bar(X[1:]-0.1, corr_probs[1:], color="g", width=0.2, align="center", label="corr")
plt.bar(X[1:]+0.1, iid_probs[1:], color="r", width=0.2, align="center", label= "iid")
plt.yscale("log")
plt.legend(loc=0)
plt.show()

In [ ]:
def normalization_agm(p, n):
    return 1 - p - (np.power(p, n+1)*(n+1) - p - n * np.power(p, n+2))/np.power(1-p, 2)

In [ ]:
corr_probs_agm = [(1-p)/normalization_agm(1-p, nq)] + [np.power(p,k)/normalization_agm(p, nq) for k in range(1,nq+1)]

In [ ]:
#fig = plt.figure()
X = np.arange(nq+1)
plt.bar(X-0.1, corr_probs_agm, color="g", width=0.2, align="center", label="corr")
plt.bar(X+0.1, iid_probs, color="r", width=0.2, align="center", label= "iid")
plt.yscale("log")
plt.legend(loc=0)
plt.show()

In [ ]:
def normalization_gm(p, n):
    return (1+p/(1-p)*(1-np.power(p,n)))

In [ ]:
import numpy as np
from define.QECCLfid.ptm import get_Pauli_tensor
from define.QECCLfid.utils import Kron
from define import globalvars as gv
from define import qcode as qc

In [ ]:
qcode = qc.QuantumErrorCorrectingCode("Steane")
qc.Load(qcode)

In [ ]:
L = [1,1,3,2,1]
np.allclose(get_Pauli_tensor(L).reshape(2**len(L),2**len(L)), Kron(*gv.Pauli[L]))

In [ ]:
nstabs = 64
nlogs = 4
# nstabs * nlogs
qc.GetOperatorsForTLSIndex(qcode, range(64, 66))

In [ ]:
qc.GenerateGroup(qcode.L)

In [ ]:
qcode.L

In [ ]:
qcode.S

In [1]:
import numpy as np
from scipy import linalg as linalg
import random
# from define.randchans import RandomUnitary
from define import qcode as qc
qcode = qc.QuantumErrorCorrectingCode("Steane")
qc.Load(qcode)
qc.PrepareSyndromeLookUp(qcode)

In [2]:
from scipy.linalg.lapack import zgeev

In [3]:
def HermitianConjugate(M):
    return M.conj().T

In [12]:
def StineToKraus(U):
    # Compute the Krauss operators for the input quantum channel, which is represented in the Stinespring dialation
    # The Krauss operator T_k is given by: <a|T_k|b> = <a e_k|U|b e_0> , where {|e_i>} is a basis for the environment and |a>, |b> are basis vectors of the system
    # Note that a k-qubit channel needs to be generated from a unitary matrix on 3*k qubits where 2*k qubits represent the environment.
    nq = int(np.log2(U.shape[0]))//3
    environment = np.eye(4**nq)[:,:,np.newaxis]
    system = np.eye(2**nq)[:,:,np.newaxis]
    krauss = np.zeros((4**nq, 2**nq, 2**nq), dtype=np.complex128)
    krauss_fast = np.zeros((4**nq, 2**nq, 2**nq), dtype=np.complex128)
    for ki in range(4**nq):
        ## The Krauss operator T_k is given by: <a|T_k|b> = <a e_k|U|b e_0>.
        for ri in range(2**nq):
            for ci in range(2**nq):
                leftProduct = np.dot(HermitianConjugate(np.kron(system[ri, :, :], environment[ki, :, :])), U)
                lv = HermitianConjugate(np.kron(system[ri, :, :], environment[ki, :, :]))
                fast_lv = np.where(np.arange(8**nq) == ri * 4**nq + ki, 1, 0)[np.newaxis, :]
                print("|lv - fast_lv| = {}".format(np.sum(lv - fast_lv)))
                fast_row = U[np.newaxis, ri * 4**nq + ki, :]
                print("|leftProduct - fast_row| = {}".format(np.linalg.norm(leftProduct - fast_row)))
                krauss[ki, ri, ci] = np.dot(
                    leftProduct, np.kron(system[ci, :, :], environment[0, :, :])
                )[0, 0]
                krauss_fast[ki, ri, ci] = U[ri * 4**nq + ki, ci * 4**nq]
                print("krauss - fast_krauss = {}".format(abs(krauss[ki, ri, ci] - krauss_fast[ki, ri, ci])))
    return (krauss, krauss_fast)

In [5]:
def FastStineToKraus(U):
    # Compute the Krauss operators for the input quantum channel, which is represented in the Stinespring dialation
    # The Krauss operator T_k is given by: <a|T_k|b> = <a e_k|U|b e_0> , where {|e_i>} is a basis for the environment and |a>, |b> are basis vectors of the system
    # Note that a k-qubit channel needs to be generated from a unitary matrix on 3*k qubits where 2*k qubits represent the environment.
    nq = int(np.log2(U.shape[0]))//3
    # environment = np.eye(4**nq)[:,:,np.newaxis]
    # system = np.eye(2**nq)[:,:,np.newaxis]
    krauss = np.zeros((4**nq, 2**nq, 2**nq), dtype=np.complex128)
    for ki in range(4**nq):
        ## The Krauss operator T_k is given by: <a|T_k|b> = <a e_k|U|b e_0>.
        for ri in range(2**nq):
            for ci in range(2**nq):
                #leftProduct = np.dot(HermitianConjugate(np.kron(system[ri, :, :], environment[ki, :, :])), U)
                #lv = HermitianConjugate(np.kron(system[ri, :, :], environment[ki, :, :]))
                #fast_lv = np.where(np.arange(8**nq) == ri * 4**nq + ki, 1, 0)[np.newaxis, :]
                #print("|lv - fast_lv| = {}".format(np.sum(lv - fast_lv)))
                #fast_row = U[np.newaxis, ri * 4**nq + ki, :]
                #print("|leftProduct - fast_row| = {}".format(np.linalg.norm(leftProduct - fast_row)))
                #krauss[ki, ri, ci] = np.dot(leftProduct, np.kron(system[ci, :, :], environment[0, :, :]))[0, 0]
                krauss[ki, ri, ci] = U[ri * 4**nq + ki, ci * 4**nq]
                #print("krauss - fast_krauss = {}".format(abs(krauss[ki, ri, ci] - U[ri * 4**nq + ki, ci * 4**nq])))
    return krauss

In [6]:
def RandomUnitary(prox, dim):
    # Generate a random unitary matrix which is close to Identity.
    M = np.random.standard_normal(size=(dim, dim)) + 1j * np.random.standard_normal(size=(dim, dim))
    H = (M + M.conj().T) / np.longdouble(2)
    U = linalg.expm(1j * prox * H)
    return U

In [7]:
nq = 1
prox = 0.1
U = RandomUnitary(prox, 8**nq)

In [13]:
(K, FK) = StineToKraus(HermitianConjugate(U))

|lv - fast_lv| = 0.0
|leftProduct - fast_row| = 0.0
krauss - fast_krauss = 0.0
|lv - fast_lv| = 0.0
|leftProduct - fast_row| = 0.0
krauss - fast_krauss = 0.0
|lv - fast_lv| = 0.0
|leftProduct - fast_row| = 0.0
krauss - fast_krauss = 0.0
|lv - fast_lv| = 0.0
|leftProduct - fast_row| = 0.0
krauss - fast_krauss = 0.0
|lv - fast_lv| = 0.0
|leftProduct - fast_row| = 0.0
krauss - fast_krauss = 0.0
|lv - fast_lv| = 0.0
|leftProduct - fast_row| = 0.0
krauss - fast_krauss = 0.0
|lv - fast_lv| = 0.0
|leftProduct - fast_row| = 0.0
krauss - fast_krauss = 0.0
|lv - fast_lv| = 0.0
|leftProduct - fast_row| = 0.0
krauss - fast_krauss = 0.0
|lv - fast_lv| = 0.0
|leftProduct - fast_row| = 0.0
krauss - fast_krauss = 0.0
|lv - fast_lv| = 0.0
|leftProduct - fast_row| = 0.0
krauss - fast_krauss = 0.0
|lv - fast_lv| = 0.0
|leftProduct - fast_row| = 0.0
krauss - fast_krauss = 0.0
|lv - fast_lv| = 0.0
|leftProduct - fast_row| = 0.0
krauss - fast_krauss = 0.0
|lv - fast_lv| = 0.0
|leftProduct - fast_row| = 0.0


In [14]:
K

array([[[ 9.69096865e-01-0.05181549j, -2.69876663e-02-0.05593586j],
        [ 1.01984210e-02-0.05411548j,  9.78267175e-01+0.03240107j]],

       [[-6.06278516e-04+0.05601989j, -2.37393313e-03-0.06314838j],
        [ 4.77317838e-02-0.05999135j,  4.78043361e-02-0.02793563j]],

       [[-1.21895084e-01+0.09342002j, -9.21801965e-02+0.05746575j],
        [-4.89984269e-02-0.03968673j,  4.15260476e-02-0.07659863j]],

       [[-9.95374111e-02+0.09127437j,  4.85498025e-02+0.00711862j],
        [-6.70815428e-03-0.01651505j, -8.93192980e-02-0.0353608j ]]])

In [16]:
FK-K

array([[[0.+0.j, 0.+0.j],
        [0.+0.j, 0.+0.j]],

       [[0.+0.j, 0.+0.j],
        [0.+0.j, 0.+0.j]],

       [[0.+0.j, 0.+0.j],
        [0.+0.j, 0.+0.j]],

       [[0.+0.j, 0.+0.j],
        [0.+0.j, 0.+0.j]]])

In [ ]:
# # Constants:
# #     K -- cutoff length = 3
# """
# 1. Sample k from the Poisson distribution with mean = 1. If k is larger than K, sample again.
# 2. Given a k, choose a random subset of {1, 2, ..., qcode.N} of size k.
# 3. Generate Kraus operators for a k-qubit quantum channel and assign the supports from the random subset chosen above.
# 4. We form a dictionary with supports and the respective Kraus operators, and give it to multi_qubit_kraus.
# """

In [ ]:
def SamplePoisson(mean, cutoff=None):
    # Sample a number from a Poisson distribution.
    # If the random variable takes a value above a cutoff, sample again until it returns a value below the cutoff.
    if (cutoff == None):
        rv_val = np.random.poisson(lam = mean)
    else:
        rv_val = cutoff + 1
        while (rv_val > cutoff):
            rv_val = np.random.poisson(lam = mean)
    return rv_val

In [ ]:
[SamplePoisson(1, 3) for __ in range(10)]

In [ ]:
def get_kraus_poisson(rotation_angle, qcode, cutoff = 3, n_maps = 3):
    r"""
    Sub-routine to prepare the dictionary for error eps = sum of cptp maps
    Generates kraus by using stine to kraus
    of random multi-qubit unitary operators rotated by rotation_angle/2**|support|
    Probability associated with each weight-k error scales exponentially p_error^k (except weights <= w_thresh)
    Input :
    rotation_angle = the angle to rotate the Stinespring unitary by
    qcode = QEC
    n_maps = number of maps to be considered for the sum
    Returns :
    dict[key] = (support,krauslist)
    where key = number associated to the operation applied (not significant)
    support = tuple describing which qubits the kraus ops act on
    krauslist = krauss ops acting on support
    """
    kraus_dict = {}
    prob_maps = np.array([1/n_maps]*n_maps) # Probabilites associated to the above CPTP maps
    cutoff = 3 # Cutoff number of qubits for poissson distribution
    cptp_map_count = 0
    for __ in range(n_maps):
        n_q = SamplePoisson(mean = 1, cutoff=cutoff)
        support = tuple(sorted((random.sample(range(qcode.N), n_q))))
        if n_q == 0:
            rand_unitary = 1.0
            kraus_dict[cptp_map_count] = (support,[rand_unitary])
        else:
            rand_unitary = RandomUnitary(
                rotation_angle/(2**(3*n_q)), 2**(3*n_q)
            )
            kraus_dict[cptp_map_count] = (support, StineToKraus(rand_unitary))
        cptp_map_count += 1
    
    # Multiplying kraus by their respective probabilities
    for key, (support, krauslist) in kraus_dict.items():
        for k in range(len(krauslist)):
            # print("k = {}".format(k))
            kraus_dict[key][1][k] *= np.sqrt(prob_maps[key])
    return kraus_dict


In [ ]:
dict_k = get_kraus_poisson(0.1, qcode, cutoff = 4, n_maps = 3)

In [ ]:
dict_k

In [ ]:
mat = [[-3.84 + 1j *  2.25, -8.94 + 1j * -4.75,  8.95 + 1j * -6.53,  -9.87 + 1j * 4.82],[-0.66 + 1j *  0.83, -4.40 + 1j * -3.82, -3.50 + 1j * -4.26,  -3.15 + 1j * 7.36],[-3.99 + 1j * -4.73, -5.88 + 1j * -6.60, -3.36 + 1j * -0.40,  -0.75 + 1j * 5.23],[7.74 + 1j *  4.18,  3.66 + 1j * -7.53,  2.58 + 1j *  3.60,   4.59 + 1j * 5.41]]
M = np.array(mat, dtype=np.complex128)

In [ ]:
np.round(M, 5)

In [ ]:
# (np_eval, np_evecs) = np.linalg.eig(M)
(np_eval, __, np_evecs, info) = zgeev(M)

In [ ]:
for i in range(4):
    print("e_%d\n%g + i %g" % (i + 1, np.real(np_eval[i]), np.imag((np_eval[i]))))
    row = ["(%g + i %g)" % (np.real(np_evecs[j,i]), np.imag(np_evecs[j,i])) for j in range(4)]
    print("v_%d\n%s" % (i + 1, ", ".join(row)))
    print("---")

In [ ]:
#eigvecs = np.array([[0.707106781186548 + 1j * 0.000000000000000, 0.000000000000000 + 1j * 0.000000000000000, 0.000000000000000 + 1j * 0.000000000000000, 0.707106781186547 + 1j * -0.000000000000000], [0.707106781186547 + 1j * 0.000000000000000, 0.000000000000000 + 1j * 0.000000000000000, 0.000000000000000 + 1j * 0.000000000000000, -0.707106781186547 + 1j * 0.000000000000000], [-0.001644226287961 + 1j * 0.000000000000000, -0.707104869534862 + 1j * 0.000000000000000, 0.707104869534862 + 1j * 0.000000000000000, 0.001644226287961 + 1j * 0.000000000000000], [0.000000000000000 + 1j * 0.000000000000000, 0.707106781186547 + 1j * 0.000000000000000, 0.707106781186548 + 1j * 0.000000000000000, -0.000000000000000 + 1j * 0.000000000000000]], dtype = np.complex128)
np_reconM = np.array(sum([eigvals[d] * np.dot(HermitianConjugate(eigvecs[d, np.newaxis, :]), eigvecs[d, np.newaxis, :]) for d in range(4)]), dtype=np.complex128)

In [ ]:
M - np_reconM

In [ ]:
reconstructed = [[0.494261994364053 + 1j * 0.000000000000000, 0.000000000000000 + 1j * 0.000000000000000, 0.000000000000000 + 1j * 0.000000000000000, 0.494217022439872 + 1j * 0.000000000000000], [0.000000000000000 + 1j * 0.000000000000000, 0.005738005635947 + 1j * 0.000000000000000, 0.005693033711766 + 1j * 0.000000000000000, 0.000000000000000 + 1j * 0.000000000000000], [0.000000000000000 + 1j * 0.000000000000000, 0.005693033711766 + 1j * 0.000000000000000, 0.005738005635947 + 1j * 0.000000000000000, 0.000000000000000 + 1j * 0.000000000000000], [0.494217022439872 + 1j * 0.000000000000000, 0.000000000000000 + 1j * 0.000000000000000, 0.000000000000000 + 1j * 0.000000000000000, 0.494261994364053 + 1j * 0.000000000000000]]
c_reconM = np.array(reconstructed, dtype = np.complex128)

In [ ]:
np_reconM - c_reconM

In [ ]:
M - np_reconM

In [ ]:
matA = np.array([[0.494261994364053 + 1j * 0.000000000000000, 0.000000000000000 + 1j * 0.000000000000000, 0.000000000000000 + 1j * 0.000000000000000, 0.494217022439872 + 1j * 0.000000000000000], [0.000000000000000 + 1j * 0.000000000000000, 0.005738005635947 + 1j * 0.000000000000000, 0.005693033711766 + 1j * 0.000000000000000, 0.000000000000000 + 1j * 0.000000000000000], [0.000000000000000 + 1j * 0.000000000000000, 0.005693033711766 + 1j * 0.000000000000000, 0.005738005635947 + 1j * 0.000000000000000, 0.000000000000000 + 1j * 0.000000000000000], [0.494217022439872 + 1j * 0.000000000000000, 0.000000000000000 + 1j * 0.000000000000000, 0.000000000000000 + 1j * 0.000000000000000, 0.494261994364053 + 1j * 0.000000000000000]] ,dtype=np.complex128)
matB = np.array([[0.494261994485634 + 1j * 0.000000000000000, 0.000000052286177 + 1j * 0.000000000000000, -0.000000052286177 + 1j * 0.000000000000000, 0.494217022318291 + 1j * 0.000000000000000], [0.000000052286177 + 1j * 0.000000000000000, 0.005738005514367 + 1j * 0.000000000000000, 0.005693033833347 + 1j * 0.000000000000000, -0.000000052286177 + 1j * 0.000000000000000], [-0.000000052286177 + 1j * 0.000000000000000, 0.005693033833347 + 1j * 0.000000000000000, 0.005738005514367 + 1j * 0.000000000000000, 0.000000052286177 + 1j * 0.000000000000000], [0.494217022318291 + 1j * 0.000000000000000, -0.000000052286177 + 1j * 0.000000000000000, 0.000000052286177 + 1j * 0.000000000000000, 0.494261994485633 + 1j * 0.000000000000000]], dtype = np.complex128)

In [ ]:
matA - matB